# RAG RetrieveAndGenerate API
Chat with your document

In [1]:
%pip install --upgrade pip
%pip install --upgrade boto3
%pip install --upgrade botocore
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.
  Using cached boto3-1.34.147-py3-none-any.whl.metadata (6.6 kB)
  Using cached botocore-1.34.147-py3-none-any.whl.metadata (5.7 kB)
  Using cached s3transfer-0.10.2-py3-none-any.whl.metadata (1.7 kB)
Using cached boto3-1.34.147-py3-none-any.whl (139 kB)
Using cached botocore-1.34.147-py3-none-any.whl (12.4 MB)
Using cached s3transfer-0.10.2-py3-none-any.whl (82 kB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.33.13
    Uninstalling botocore-1.33.13:
      Successfully uninstalled botocore-1.33.13
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.8.2
    Uninstalling s3transfer-0.8.2:
      Successfully uninstalled s3transfer-0.8.2
  Attempting uninstall: boto3
    Found existing installation: boto3 1.33.2
    Uninstalling boto3-1.33.2:
      Successfully uninstalled boto3-1.33.2
Note: you may need to restart the kernel to use updated packages.
Note: you may 

In [2]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [3]:
import boto3
boto3.__version__

'1.34.147'

Initialize client for Amazon Bedrock for accessing the `RetrieveAndGenerate` API.

In [4]:
import boto3
import pprint
from botocore.client import Config

pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})

bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              region_name=region,
                              config=bedrock_config,
                                    )
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

print(region)

us-west-2


For data, you can either upload the document you want to chat with or point to the Amazon Simple Storage Service (Amazon S3) bucket location that contains your file. We provide you with both options in the notebook. However in both cases, the supported file formats are PDF, MD (Markdown), TXT, DOCX, HTML, CSV, XLS, and XLSX. Make that the file size does not exceed 10 MB and contains no more than 20K tokens. A token is considered to be a unit of text, such as a word, sub-word, number, or symbol, that is processed as a single entity. Due to the preset ingestion token limit, it is recommended to use a file under 10MB. However, a text-heavy file, that is much smaller than 10MB, can potentially breach the token limit.

### Option 1 - Upload the document

In our example, we will use a pdf file.

In [13]:
# load pdf
from pypdf import PdfReader
# creating a pdf reader object
file_name = "<path of your file such as file.pdf>" #path of the file on your local machine.
reader = PdfReader(file_name)
# printing number of pages in pdf file
print(len(reader.pages))
text = ""
page_count = 1
for page in reader.pages:
    text+= f"\npage_{str(page_count)}\n {page.extract_text()}"
print(text)

FileNotFoundError: [Errno 2] No such file or directory: '<path of your file such as file.pdf>'

### Option 2 - Point to S3 location of your file
Make sure to replace the `bucket_name` and `prefix_file_name` to the location of your file.

In [5]:
bucket_name = "bedrock-doit-demo"
prefix_file_name = "rag/AMZN-2022-Shareholder-Letter.pdf" 
document_s3_uri = f's3://{bucket_name}/{prefix_file_name}'


### RetreiveAndGenerate API for chatting with your document
The code in the below cell, defines a Python function called `retrieveAndGenerate` that takes two optional arguments: `input` (the input text) and `sourceType` (the type of source to use, defaulting to "S3"). It also sets a default value for the `model_id` parameter.

The function constructs an Amazon Resource Name (ARN) for the specified model using the `model_id` and the `REGION` variable.

If the `sourceType` is "S3", the function calls the `retrieve_and_generate` method of the `bedrock_agent_client` object, passing in the input text and a configuration for retrieving and generating from external sources. The configuration specifies that the source is an S3 location, and it provides the S3 URI of the document.

If the `sourceType` is not "S3", the function calls the same `retrieve_and_generate` method, but with a different configuration. In this case, the source is specified as byte content, which includes a file name, content type (application/pdf), and the actual text data.

In [6]:
def retrieveAndGenerate(input, sourceType="S3", model_id = "anthropic.claude-3-sonnet-20240229-v1:0"):
    model_arn = f'arn:aws:bedrock:{region}::foundation-model/{model_id}'
    if sourceType=="S3":
        return bedrock_agent_client.retrieve_and_generate(
            input={
                'text': input
            },
            retrieveAndGenerateConfiguration={
                'type': 'EXTERNAL_SOURCES',
                'externalSourcesConfiguration': {
                    'modelArn': model_arn,
                    "sources": [
                        {
                            "sourceType": sourceType,
                            "s3Location": {
                                "uri": document_s3_uri
                            }
                        }
                    ]
                }
            }
        )
    else:
        return bedrock_agent_client.retrieve_and_generate(
            input={
                'text': input
            },
            retrieveAndGenerateConfiguration={
                'type': 'EXTERNAL_SOURCES',
                'externalSourcesConfiguration': {
                    'modelArn': model_arn,
                    "sources": [
                        {
                            "sourceType": sourceType,
                            "byteContent": {
                                "identifier": file_name,
                                "contentType": "application/pdf",
                                "data": text,
                                }
                        }
                    ]
                }
            }
        )

If you want to chat with the document by uploading the file use `sourceType` as `BYTE_CONTENT` for pointing it to s3 bucket, use `sourceType` as `S3`.

In [7]:
query = "What is AWS year-over-year revenue in 2022?"
response = retrieveAndGenerate(input=query)
generated_text = response['output']['text']
pp.pprint(generated_text)

('According to the letter, AWS had an $85 billion annualized revenue run rate '
 'in 2022, growing 29% year-over-year on a $62 billion revenue base in 2021.')


## Citations or source attributions
Lets retrieve the source attribution or citations for the above response.


In [8]:
citations = response["citations"]
contexts = []
for citation in citations:
    retrievedReferences = citation["retrievedReferences"]
    for reference in retrievedReferences:
         contexts.append(reference["content"]["text"])

pp.pprint(contexts)

[ 'Despite growing 29% year-over- year (“YoY”) in 2022 on a $62B revenue base, '
  'AWS faces short-term headwinds right now as companies are being more '
  'cautious in spending given the challenging, current macroeconomic '
  'conditions. While some companies might obsess over how they could extract '
  'as much money from customers as possible in these tight times, it’s neither '
  'what customers want nor best for customers in the long term, so we’re '
  'taking a different tack. One of the many advantages of AWS and cloud '
  'computing is that when your business grows, you can seamlessly scale up; '
  'and conversely, if your business contracts, you can choose to give us back '
  'that capacity and cease paying for it. This elasticity is unique to the '
  'cloud, and doesn’t exist when you’ve already made expensive capital '
  'investments in your own on-premises datacenters, servers, and networking '
  'gear. In AWS, like all our businesses, we’re not trying to optimize for any 

## Next Steps
In this notebook, we covered how Knowledge Bases for Amazon Bedrock now simplifies asking questions on a single document. We also demonstrated how to configure and use this capability through the Amazon Bedrock - AWS SDK, showcasing the simplicity and flexibility of this feature, which provides a zero-setup solution to gather information from a single document, without setting up a vector database.

To further explore the capabilities of Knowledge Bases for Amazon Bedrock, refer to the following resources:

[Knowledge bases for Amazon Bedrock](#https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html)
